In [1]:
import os
import pandas as pd
import dask.dataframe as dd
from knowledge_graph_generator import KnowledgeGraphGenerator
from fusion import TransE, TransEFuser

import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
dask_df = dd.read_csv("data/relation_files/*.csv")
full_df = dask_df.compute()

fuser = TransEFuser(embed_dim=3, max_epochs=2, n_splits=3)
kg_df = fuser.fuse(full_df)

relation features the miscellaneous theme, cv 0, epoch 0, loss 3.607142857142857
relation features the miscellaneous theme, cv 0, epoch 1, loss 3.392857142857143
relation features the miscellaneous theme, cv 1, epoch 0, loss 2.9464285714285716
relation features the miscellaneous theme, cv 1, epoch 1, loss 3.25
relation features the miscellaneous theme, cv 2, epoch 0, loss 2.767857142857143
relation features the miscellaneous theme, cv 2, epoch 1, loss 4.0
relation features the theme, cv 0, epoch 0, loss 3.4546204620462047
relation features the theme, cv 0, epoch 1, loss 3.4653465346534653
relation features the theme, cv 1, epoch 0, loss 2.7797029702970297
relation features the theme, cv 1, epoch 1, loss 2.7268976897689767
relation features the theme, cv 2, epoch 0, loss 2.4174917491749173
relation features the theme, cv 2, epoch 1, loss 2.4174917491749173
relation is, cv 0, epoch 0, loss 3.3429203539823007
relation is, cv 0, epoch 1, loss 3.6393805309734515
relation is, cv 1, epoch 0, 

In [6]:
filtered_df = kg_df.loc[kg_df["probability"] > 0.75].reset_index(drop=True)
x = filtered_df[filtered_df["entity_id"] == "m/spiderman_2"]
for i in range(len(filtered_df[filtered_df["entity_id"] == "m/spiderman"])):
    r = x.iloc[i]
    print(r["relation"], r["value"], r["probability"])

96
features the theme these actors 0.8937830291416001
features the theme raging hormones who learning 0.956212739666196
features the theme the special effects 0.859557922791379
features the theme your jaw 0.9607189669955802
features the theme a good bit of harmless fun 0.7659165982224327
features the theme superhero movie 0.8005689801205209
features the theme the characters 0.8959785323063019
features the theme charisma 0.7859395262592798
features the theme an innocence 0.833366928861863
features the theme execution 0.8741530540379767
features the theme spiderman 0.9322083702765583
features the theme youth acquired a reputation as the superpower 0.9318223795897311
features the theme a whole movie about the hero 0.8751199524167534
features the theme an already announced sequel 0.9007807805174253
features the theme flicks 0.9381188582717788
features the theme the whole thing 0.9492898737282399
features the theme the spirit of its source material 0.802405313421382
features the theme top o

IndexError: single positional indexer is out-of-bounds